In [ ]:
import dotenv
import os
import re
import os
import glob

dotenv.load_dotenv()

OPENAI_KEY = os.environ.get("OPENAI_API_KEY")

## Question and answer prompt generator

In [ ]:
import pandas as pd

q_and_a_df = pd.read_csv("A_111_1_vF for Mark - Experiment Level.csv")

q_and_a_df["How to code"] = q_and_a_df["How to code"].replace(
    {"Y/N": "Y/N (yes or no)", "Y/N/Unclear": "Y/N/Unclear (Yes, No or Unclear)"}
)

In [ ]:
for label, question, answer_desc in q_and_a_df[["Variable", "Description", "How to code"]].head(5).values:
    print(f"Label: {label}, Question: {question}, Answer: {answer_desc}")

-----------------------
## Markdown split and QA

In [ ]:
q_and_a_behavior_df = pd.read_csv('behavior_level.csv')

q_and_a_experiment_df = pd.read_csv('experimental_level.csv')

q_and_a_behavior_df.columns

In [ ]:
for vars in q_and_a_experiment_df['Variable name'].unique():
    if vars in article_level:
        print(f"{vars} - Article level")


In [ ]:


masterdb_df['Paper_Exp_ID_better'] = masterdb_df.Paper_Exp_ID.str.extract(r'([A-Z]\_\d+)')

In [ ]:
[v for v in article_level if v not in relevant_columns]
article_level


In [ ]:
import json

article_level = []
experiment_level = []

for colomn in masterdb_df.columns:
    if (
        masterdb_df.groupby("Paper_Exp_ID_better")[colomn]
        .nunique()
        .reset_index(name="unique_count")
        .sort_values(["unique_count"], ascending=False)
        .max()["unique_count"]
        == 1
    ):
        article_level.append(colomn)
    else:
        experiment_level.append(colomn)

print(f"{len(article_level)} - Article level: {article_level} ")
print(f"{len(experiment_level)} - Experiment level: {experiment_level}")

In [ ]:
for at in article_level:
    print(at)

In [ ]:
relevant_columns = []

for vars in q_and_a_df.Variable.unique():
    if vars in article_level:
        print(f"{vars} - Article level")
        relevant_columns.append(vars)

len(relevant_columns)

In [ ]:
masterdb_df = pd.read_csv('MasterDB_Full.csv')

pattern = r'([A-Z]\_\d+)'

available_ids = []

for id in list(masterdb_df.groupby('Paper_Exp_ID').count().index):
    match = re.search(pattern, id)
    if match:
        extracted = match.group(1)
        available_ids.append(extracted)
    else:
        print("No match found.")

len(available_ids)

In [ ]:

available_markdown_files = []

folder_path = "markdown_articles"
markdown_pattern = "*.mmd"
markdown_files = glob.glob(os.path.join(folder_path, markdown_pattern))

pattern = r'[^/]+/([A-Z0-9_]+_[0-9]+[a-zA-Z]*_[0-9]+)'
pattern_id = r'([A-Z]\_\d+)'

for markdown_file in markdown_files:
    # print("Processing:", markdown_file)

    # with open(markdown_file, 'r', encoding='utf-8') as file:
    #     markdown_content = file.read()
    
    # file.close()
    match = re.search(pattern_id, markdown_file)
    if match:
        code = match.group(1)
        if code in available_ids:
            # print("Found:", code)
            available_markdown_files.append(markdown_file)
    else:
        print("No match found.")


file_name = "markdown_articles/A_1_2022_ADigitalApproach.mmd"


len(available_markdown_files)

In [ ]:
def get_label(file_name):
    # pattern = r'[^/]+/([A-Z0-9_]+_[0-9]+[a-zA-Z]*_[0-9]+)'
    pattern = r'([A-Z]\_\d+)'
    match = re.search(pattern, file_name)
    if match:
        code = match.group(1)
        return code
    else:
        return None

In [ ]:
masterdb_df[masterdb_df['Paper_Exp_ID'].str.contains('A_114')]

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


def create_vector_db(file_name):
    loader = TextLoader(file_name)
    markdown_doc = loader.load()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(markdown_doc[0].page_content)

    # Define the Text Splitter

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

    # Create a split of the document using the text splitter
    splits = text_splitter.split_documents(md_header_splits)


    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

    persist_directory = "./docs/chroma/"

    # Create the vector store
    vectordb = Chroma.from_documents(
        documents=splits, embedding=embedding#, persist_directory=persist_directory
    )

    return vectordb

    # print(vectordb._collection.count())

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_answer_format(vectordb, question, answer):
    
    # Build prompt
    template = """Use the following pieces of context to answer the question at the end according to the format of the answer provided. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
    {context}
    Question: {question}, Answer format: """+answer+""",
    Answer:"""

    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # QA_CHAIN_PROMPT = PromptTemplate(
    #     template=template, input_variables=["context", "question", "answer_format"]
    # )

    llm = ChatOpenAI(model_name="gpt-4-32k", temperature=0)

    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(
            search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
        ),
        chain_type="stuff",
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

    result = qa_chain({"query": question})

    # return result
    return result["result"]

In [ ]:
from langchain.callbacks import get_openai_callback

# print(question)
# print(answer_desc)

with get_openai_callback() as cb:
    vectordb = create_vector_db("sample_paper/A_19_2022_DoHonestyNudges.mmd")
    print(get_answer_format(vectordb, 'how is the paper', 'free text'))
    print(f"Prompt Tokens: ${cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [ ]:
answer_dic = {"label": [], "gpt_answer": []}

for label, question, answer_desc in (
    q_and_a_df[["Variable", "Description", "How to code"]].head(25).values
):
    print(f"Label: {label}, Answer: {answer_desc}")
    answer_dic["label"].append(label)
    answer_dic["gpt_answer"].append(get_answer_format(vectordb, question, answer_desc))

In [ ]:
answer_df = pd.DataFrame(answer_dic)
answer_df["human_answer"] = ""
answer_df

In [ ]:
all_dataframes_others = []

In [ ]:
for md_file_name in available_markdown_files[:20]:
    # get the label id
    label_id = get_label(md_file_name)
    # seperate the answered section for that id
    md_df = masterdb_df[masterdb_df["Paper_Exp_ID"].str.contains(label_id)]

    # create a vector db for the article text
    file_vector_db = create_vector_db(md_file_name)

    answer_dic = {"label": [], "gpt_answer": [], "prompt_tokens": [], "completion_tokens": [], "total_cost": []}

    for label, question, answer_desc in (
        # get the relevant rows from the q_and_a_df
        q_and_a_df[q_and_a_df['Variable'].isin(relevant_columns)][["Variable", "Description", "How to code"]].values
    ):
        print(f"Label: {label}, Answer: {answer_desc}")
        answer_dic["label"].append(label)
        with get_openai_callback() as cb:
            chat_result = get_answer_format(file_vector_db, question, answer_desc)
            answer_dic["gpt_answer"].append(chat_result)
            answer_dic["prompt_tokens"].append(cb.prompt_tokens)
            answer_dic["completion_tokens"].append(cb.completion_tokens)
            answer_dic["total_cost"].append(cb.total_cost)

    answer_df = pd.DataFrame(answer_dic)
    answer_df["human_answer"] = ""

    # take the first row of the md_df and add as human answer
    my_dict = md_df.iloc[0].to_dict()

    for key, value in my_dict.items():
        mask = answer_df["label"] == key
        answer_df.loc[mask, "human_answer"] = value

    answer_df["article_id"] = label_id

    all_dataframes.append(answer_df)

In [ ]:
all_dataframes_others = []

for md_file_name in available_markdown_files[:20]:
    # get the label id
    label_id = get_label(md_file_name)
    # seperate the answered section for that id
    md_df = masterdb_df[masterdb_df["Paper_Exp_ID"].str.contains(label_id)]

    # create a vector db for the article text
    file_vector_db = create_vector_db(md_file_name)

    answer_dic = {"label": [], "gpt_answer": [], "prompt_tokens": [], "completion_tokens": [], "total_cost": []}

    for label, question, answer_desc in (
        # get the relevant rows from the q_and_a_df
        q_and_a_df[["Variable", "Description", "How to code"]].head(25).values
    ):
        print(f"Label: {label}, Answer: {answer_desc}")
        answer_dic["label"].append(label)
        with get_openai_callback() as cb:
            chat_result = get_answer_format(file_vector_db, question, answer_desc)
            answer_dic["gpt_answer"].append(chat_result)
            answer_dic["prompt_tokens"].append(cb.prompt_tokens)
            answer_dic["completion_tokens"].append(cb.completion_tokens)
            answer_dic["total_cost"].append(cb.total_cost)

    answer_df = pd.DataFrame(answer_dic)
    answer_df["human_answer"] = ""

    # take the first row of the md_df and add as human answer
    my_dict = md_df.iloc[0].to_dict()

    for key, value in my_dict.items():
        mask = answer_df["label"] == key
        answer_df.loc[mask, "human_answer"] = value

    answer_df["article_id"] = label_id

    all_dataframes_others.append(answer_df)

In [ ]:
result_df = pd.concat(all_dataframes_others, ignore_index=True)
result_df

In [ ]:
import numpy as np

result_df.gpt_answer.replace({"Yes": "Y", "No": "N", "": "NA"}, inplace=True)

In [ ]:
result_df.head(25)

In [ ]:
result_df.fillna("NA", inplace=True)
result_df['comparison'] = result_df.apply(lambda x: x['gpt_answer'] == x['human_answer'], axis=1)
result_df

In [ ]:
result_df.iloc[2].gpt_answer

In [ ]:
result_df.groupby('article_id').sum()['total_cost']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")


colors = ["red", "green"]


plot = sns.relplot(
    x="article_id",
    y="label",
    hue="comparison",
    style="comparison",
    markers="s",
    s=3000,
    height=15,
    data=result_df,
    palette=colors,  # Set the color palette
)


plot._legend.remove()


custom_legend_labels = ["False", "True"]
for color, label in zip(colors, custom_legend_labels):
    plt.scatter([], [], c=color, marker="s", label=label)  # Use 's' for squares


plot.ax.set_aspect("equal")


plt.show()

In [ ]:
def highlight_truevalues(comp):
    # is_max = pd.Series(data=False, index=s.index)
    # is_max[column] = s.loc[column] == True
    # return ['background-color: green' if is_max.any() else 'background-color: red' for v in is_max]
    if comp:
        return 'background-color: green'
    else:
        return 'background-color: red'

result_df.style.applymap(highlight_truevalues, subset=['comparison'])


In [ ]:
result_df.groupby('article_id')['comparison'].value_counts().unstack(fill_value=0).reset_index()

-----------------------------------------------------------------------------

## Exploring an behavioral and experimental level questions

looping through parts of a conceptual experiment structure

In [ ]:
## setup question retrieval

condition_df = pd.read_csv('experimental_level.csv')

In [ ]:
from langchain.document_loaders import UnstructuredXMLLoader

loader = UnstructuredXMLLoader(
    "A_140a_2020_PromotingVoterRegistration.xml",
    mode="elements",
    strategy="fast",
)
docs = loader.load()

# markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# md_header_splits = markdown_splitter.split_text(markdown_doc[0].page_content)

# # Define the Text Splitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

# # Create a split of the document using the text splitter
# splits = text_splitter.split_documents(md_header_splits)


embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

# Create a split of the document using the text splitter
splits = text_splitter.split_documents(docs)

# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    collection_name="A_140a_2020_PromotingVoterRegistration"
    #   persist_directory=persist_directory
)


vectordb

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


def create_vector_db(file_name):
    loader = TextLoader(file_name)
    markdown_doc = loader.load()

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(markdown_doc[0].page_content)

    # Define the Text Splitter

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

    # Create a split of the document using the text splitter
    splits = text_splitter.split_documents(md_header_splits)


    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

    persist_directory = "./docs/chroma/"

    # Create the vector store
    vectordb = Chroma.from_documents(
        documents=splits, embedding=embedding#, persist_directory=persist_directory
    )

    return vectordb


In [ ]:
A_140_db = create_vector_db('markdown_articles/A_111a_2021_UsingFreshStarts.mmd')

In [ ]:
A_140_db

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_answer_format(vectordb, question, answer):
    
    # Build prompt
    template = """Use the following pieces of context to answer the question at the end according to the format of the answer provided. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
    {context}
    Question: {question}, Answer format: """+answer+""",
    Answer:"""

    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


    llm = ChatOpenAI(model_name="gpt-4", temperature=0)

    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(
            search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
        ),
        chain_type="stuff",
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

    result = qa_chain({"query": question})

    # return result
    return result["result"]

In [ ]:
condition_df.iloc[1]

In [ ]:
get_answer_format(A_140_db, condition_df.iloc[0]['Explanation'], condition_df.iloc[0]['Coding scheme'])

In [ ]:


multiline_text = """
You answered, {answer} regarding the question, {question_1} about the given document.

here is a follow up question: {question_2}
"""

# Define variables to replace placeholders
variables = {
    "answer": "Birthday Framing",
    "question_1": "Give each condition a one or two word name to describe it. Author's words: Where possible, use the label the research authors give it. Look at tables/figures to see their naming conventions for the conditions and use those if they exist.?",
    "question_2": condition_df.iloc[1]['Explanation']
}

# Use string formatting to replace placeholders with variables
formatted_text = multiline_text.format(**variables)

get_answer_format(A_140_db, formatted_text, condition_df.iloc[1]['Coding scheme'])

In [ ]:
sentence_one = "The 'Holiday Framing Condition' refers to a method used in the study where the time delay for a certain action (in this case, retirement plan contributions) was framed in reference to a holiday. For example, instead of telling an employee that they would start contributing to their retirement plan in three months, they would be told that they would start contributing after a specific holiday, such as New Year\'s or the first day of spring. The study included several different holiday framing conditions, including Thanksgiving, Martin Luther King Day, Valentine\'s Day, New Year\'s, and the first day of spring. The purpose of this framing was to investigate whether associating the start of contributions with a significant temporal landmark like a holiday would influence the likelihood of employees taking up the delayed savings opportunity."

sentence_two = "given the opportunity to sign up to save/increase savings either immediately or on a given future date in the next year -- the future date is described in terms of holidays that are not associated with beginning something new (Thanksgiving, MLK Day, and Valentine's Day) - cohort 1 only"

sentence_similarity_gpt4(sentence_one, sentence_two)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(model_name="gpt-4", temperature=0)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    A_140_db.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}),
    chain_type="stuff",
    memory=memory,
)

In [ ]:
template = (
    """Use the following pieces of context to answer the question at the end according to the format of the answer provided. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
{context}
Question: {question}, Answer format: """
    +  condition_df.iloc[0]['Coding scheme']
    + """,
Answer:"""
)

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
query = condition_df.iloc[0]['Explanation']
result = qa({"question": query})
result

In [ ]:
result['answer']

In [ ]:
query2 = "For the condition 2. Holiday Framing Condition " + condition_df.iloc[1]["Explanation"]
result = qa({"question": query2})
result

In [ ]:
query3 = "For the condition 'Holiday Framing Condition' " + condition_df.iloc[2]["Explanation"]
result = qa({"question": query3})
result

----------------------------
## Behavior

In [ ]:
behavior_df = pd.read_csv('behavior_level.csv')

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(model_name="gpt-4", temperature=0)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    A_140_db.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}),
    chain_type="stuff",
    memory=memory,
)

multiline_text = (
    """ Question: """
    + behavior_df.iloc[0]["Explanation"]
    + """, Answer format: """
    + behavior_df.iloc[0]["Coding scheme"]
)

result = qa({"question": multiline_text})
result

In [ ]:
masterdb_df.groupby('Paper_Exp_ID').nunique()

In [ ]:
masterdb_df.groupby('Paper_Exp_ID').nunique()[list(condition_df['Variable name'])]

In [ ]:
# keywords_list = [
#     "condition_financial",
#     "condition_educational",
#     "condition_forbid",
#     "condition_channel_snailmail",
#     "condition_channel_email",
#     "condition_channel_paperlive",
#     "condition_channel_inperson",
#     "condition_channel_mobileapp",
#     "condition_channel_online",
#     "condition_channel_sms",
#     "condition_channel_phone",
#     "condition_channel_nothing"
# ]

keywords_list= ["behavior_priority",
"behavior_focal",
"behavior_focal_estimate",
"behavior_metric",
"behavior_selfreport",
"financial_reqt",
"travel_reqt",
"planning_reqt",
"location_reqt",
"timing_reqt",
"prosociality",
"delay",
"authority"]

masterdb_df.groupby('Paper_Exp_ID').nunique()[keywords_list]

In [ ]:
# Calculate the count of unique entries in each column from keywords_list for each group
unique_counts = masterdb_df.groupby('Paper_Exp_ID')[keywords_list].nunique()

# Count the number of columns where unique count is greater than 1 for each group
num_columns_with_unique_count_greater_than_1 = (unique_counts > 1).sum(axis=1)

# Filter the groups where more than 4 columns have unique count greater than 1
result = unique_counts[num_columns_with_unique_count_greater_than_1 > 3]

result.index

In [ ]:
result

------------------------------
## Custom prompt per variable

In [ ]:
import json


q_and_a_behavior_df = pd.read_csv("behavior_level.csv")

q_and_a_experiment_df = pd.read_csv("experimental_level.csv")


template = """Use the following pieces of context to answer the question at the end according to the format of the answer provided. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
        {context}
        Question: {question}, Answer format: {{answer}}
        Answer:"""

output_dic_prompt = {}

output_dic_prompt[q_and_a_behavior_df.loc[0, "Variable name"]] = template.format(
    question=q_and_a_behavior_df.loc[0, "Explanation"],
    context="",
)

json.dumps(output_dic_prompt)

with open("behavior_prompt.json", "w") as outfile:
    json.dump(output_dic_prompt, outfile)

In [ ]:
q_and_a_behavior_df

## Prompt engineer

In [ ]:
import os
import openai
import csv
import json
from time import sleep
import pandas as pd


openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")


def sentence_similarity_gpt4(question, gpt_answer, human_answer):
    systemPromt = "You are a bot that compares the similarity of an answer between gpt answer and human answer to the same question, you take the\
        context of them into consideration and you will only return the score which is between 0 and 1. You are given the question, the desired answer and the gpt answer."
    userPrompt = "Question: {question} \nGPT answer: {A}\nHuman answer: {B}".format(
        question=question, A=gpt_answer, B=human_answer
    )

    functions = [
        {
            "name": "get_similarity_score",
            "description": "get the similarity between two sentences",
            "parameters": {
                "type": "object",
                "properties": {
                    "score": {
                        "type": "number",
                        "description": "the similarity score between two sentences between 0 and 1",
                    },
                },
                "required": ["score"],
            },
        }
    ]
    completion = openai.ChatCompletion.create(
        # model="gpt-4",
        model="gpt-4",
        messages=[
            {"role": "system", "content": systemPromt},
            {"role": "user", "content": userPrompt},
        ],
        temperature=0,
        functions=functions,
        function_call="auto",
    )

    return completion

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback


class ArticleQA:
    def __init__(self, vector_db, model="gpt-4"):
        self.model = model
        self.vector_db = vector_db

    def query_context(self, question, answer_format):
        # Build prompt
        template = (
            """Use the following pieces of context to answer the question at the end according to the format of the answer provided. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
        {context}
        Question: {question}, Answer format: """
            + answer_format
            + """,
        Answer:"""
        )

        QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

        llm = ChatOpenAI(model_name=self.model, temperature=0)

        qa_chain = RetrievalQA.from_chain_type(
            llm,
            retriever=self.vector_db.as_retriever(
                search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
            ),
            chain_type="stuff",
            return_source_documents=True,
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        )

        answer_dic = {}

        with get_openai_callback() as cb:
            result = qa_chain({"query": question})
            answer_dic["gpt_answer"] = result
            answer_dic["prompt_tokens"] = cb.prompt_tokens
            answer_dic["completion_tokens"] = cb.completion_tokens
            answer_dic["total_cost"] = cb.total_cost

        return answer_dic

In [ ]:
A_140_qa = ArticleQA(create_vector_db('markdown_articles/A_111a_2021_UsingFreshStarts.mmd'))

In [ ]:
s_r = sentence_similarity_gpt4(initial_prompt, "Control, Holiday Framing, Birthday Framing", h_answer)

json.loads(s_r['choices'][0]['message']['function_call']['arguments'])['score']

# 1 - mse(predicted, actual) / mse(mean of group, actual)

In [ ]:
def improve_prompt(
    article_class, prompt, prompt_answer_format, desired_prompt_answer, history=None
):
    if history is None:
        messages = [
            {
                "role": "system",
                "content": "You are a prompt engineer that improves the accuracy of the prompt. You give the user a prompt and recieve a prompt performance score between 0 and 1 on the prompt you made. Then according to the score, you change the prompt to improve the score returning only the improved prompt.",
            },
            {"role": "assistant", "content": prompt},
        ]
    else:
        messages = history

    q_and_a = article_class.query_context(prompt, prompt_answer_format)

    q_and_a["gpt_answer"]["result"]

    print("document prompt:", prompt)
    print("prompt answer:", q_and_a["gpt_answer"]["result"])

    sim_res = sentence_similarity_gpt4(
        prompt, q_and_a["gpt_answer"]["result"], desired_prompt_answer
    )

    similarity_score = json.loads(
        sim_res["choices"][0]["message"]["function_call"]["arguments"]
    )["score"]

    print("prompt score:", similarity_score)

    messages.append({"role": "user", "content": str(similarity_score)})

    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=0,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    messages.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

    return completion, messages

In [ ]:

h_answer = "delayed option takeup, immediate option takeup, immediate or delayed option takeup, average contribution rates (nov-june)"

initial_prompt = behavior_df.iloc[0]['Explanation']
answer_format = behavior_df.iloc[0]['Coding scheme']

histo = None


while True:
    
    r, h = improve_prompt(A_140_qa, initial_prompt, answer_format, h_answer, histo)

    print("new prompt:", r.choices[0].message.content)
    print("________________________________________________")

    initial_prompt = r.choices[0].message.content
    histo = h

    user_input = input("Do you want to continue (Y/N)? ").strip().lower()
    if user_input != 'y':
        break


In [ ]:

h_answer = "Control (C1 + C2), Holiday Placebo (C1), Holiday Placebo (C2), Fresh Start Framing (Birthday C1 and Holiday FS C1 + C2)"

initial_prompt = condition_df.iloc[0]['Explanation']
answer_format = condition_df.iloc[0]['Coding scheme']

histo = None


while True:
    
    r, h = improve_prompt(A_140_qa, initial_prompt, answer_format, h_answer, histo)

    print("new prompt:", r.choices[0].message.content)
    print("________________________________________________")

    initial_prompt = r.choices[0].message.content
    histo = h

    user_input = input("Do you want to continue (Y/N)? ").strip().lower()
    if user_input != 'y':
        break


In [ ]:
r2, h2 = improve_prompt(A_140_qa, r.choices[0].message.content, condition_df.iloc[0]['Coding scheme'], h_answer, history=h)

print("new prompt:", r2.choices[0].message.content)
print("history:", h2)